## かぼちゃの価格設定における線形回帰と多項式回帰 - レッスン3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalliによるインフォグラフィック</figcaption>


<!--![Dasani Madipalliによるインフォグラフィック](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ja.png){width="800"}-->

#### はじめに

これまでに、かぼちゃの価格設定データセットから収集したサンプルデータを使って回帰分析が何であるかを探求しました。また、それを`ggplot2`を使って視覚化しました。💪

これからは、機械学習における回帰分析をさらに深く掘り下げていきます。このレッスンでは、*基本的な線形回帰*と*多項式回帰*の2種類の回帰分析について、これらの手法の基礎となる数学も含めて学びます。

> このカリキュラム全体を通じて、数学の知識が最小限であることを前提とし、他分野から来た学生にも理解しやすいようにしています。注釈、🧮 数学のヒント、図解、その他の学習ツールを活用して理解を助けます。

#### 準備

改めて、このデータを読み込む目的は、データに基づいて質問をすることです。

-   かぼちゃを買うのに最適な時期はいつですか？

-   ミニチュアかぼちゃのケースの価格はどれくらいですか？

-   半バスケットで買うべきか、それとも1 1/9バスケットの箱で買うべきか？このデータをさらに掘り下げてみましょう。

前のレッスンでは、`tibble`（データフレームの現代的な再構築）を作成し、元のデータセットの一部を標準化してバスケット単位で価格を設定しました。しかし、その方法では約400のデータポイントしか収集できず、秋の月に限定されてしまいました。データをさらにクリーニングすることで、データの性質についてもう少し詳しく知ることができるかもしれませんね。どうなるでしょうか... 🕵️‍♀️

このタスクでは、以下のパッケージが必要です：

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/)は、データサイエンスをより速く、簡単に、そして楽しくするために設計された[Rパッケージのコレクション](https://www.tidyverse.org/packages)です。

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/)フレームワークは、モデリングと機械学習のための[Rパッケージのコレクション](https://www.tidymodels.org/packages/)です。

-   `janitor`: [janitorパッケージ](https://github.com/sfirke/janitor)は、汚れたデータを調査しクリーニングするためのシンプルなツールを提供します。

-   `corrplot`: [corrplotパッケージ](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html)は、変数間の隠れたパターンを検出するのに役立つ自動変数並べ替えをサポートする相関行列の視覚的探索ツールを提供します。

以下のコマンドでこれらをインストールできます：

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

以下のスクリプトは、このモジュールを完了するために必要なパッケージがインストールされているかを確認し、欠けている場合はインストールします。


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

後でこれらの素晴らしいパッケージを読み込み、現在のRセッションで利用可能にします。（これは単なる例示のためです。`pacman::p_load()`はすでにこれを行っています）

## 1. 線形回帰直線

レッスン1で学んだように、線形回帰の目的は、以下のために*最適な直線*をプロットできるようにすることです：

-   **変数間の関係を示す**。変数間の関係を明らかにする。

-   **予測を行う**。新しいデータポイントがその直線に対してどこに位置するかを正確に予測する。

この種の直線を描くために、**最小二乗回帰**と呼ばれる統計的手法を使用します。`最小二乗`という用語は、回帰直線の周囲にあるすべてのデータポイントを二乗して合計することを意味します。理想的には、その最終的な合計が可能な限り小さいことが望ましいです。なぜなら、エラーの数を少なくしたい、つまり`最小二乗`にしたいからです。このようにして、最適な直線は、二乗誤差の合計が最も小さい値を与える直線となります。そのため、この手法は*最小二乗回帰*と呼ばれます。

これは、すべてのデータポイントからの累積距離が最小となる直線をモデル化したいからです。また、加算する前に項を二乗するのは、その方向ではなく大きさに関心があるためです。

> **🧮 数学を見てみよう**
>
> この直線、つまり*最適な直線*は[次の方程式](https://en.wikipedia.org/wiki/Simple_linear_regression)で表されます：
>
>     Y = a + bX
>
> `X`は`説明変数`または`予測子`を指します。`Y`は`従属変数`または`結果`を指します。この直線の傾きは`b`であり、`a`はY切片を指します。これは、`X = 0`のときの`Y`の値を意味します。
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "slope = $y/x$")
    ジェン・ルーパーによるインフォグラフィック
>
> まず、傾き`b`を計算します。
>
> 言い換えると、かぼちゃデータの元の質問「月ごとに1ブッシェルあたりのかぼちゃの価格を予測する」に関連して、`X`は価格を指し、`Y`は販売月を指します。
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.ja.png)
    ジェン・ルーパーによるインフォグラフィック
> 
> Yの値を計算します。もし約\$4を支払っているなら、それは4月に違いありません！
>
> この直線を計算する数学は、直線の傾きを示す必要があります。これはまた、切片、つまり`X = 0`のときの`Y`の位置にも依存します。
>
> これらの値の計算方法は、[Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html)のウェブサイトで確認できます。また、[この最小二乗計算機](https://www.mathsisfun.com/data/least-squares-calculator.html)を訪れて、数値の値が直線にどのように影響するかを観察してください。

怖くないですよね？🤓

#### 相関

もう1つ理解すべき用語は、与えられたXとY変数間の**相関係数**です。散布図を使用すると、この係数をすぐに視覚化できます。データポイントがきれいな直線に散らばっているプロットは高い相関を持ちますが、データポイントがXとYの間で無秩序に散らばっているプロットは低い相関を持ちます。

良い線形回帰モデルは、最小二乗回帰法を使用して回帰直線を引いた際に、相関係数が高い（1に近い）ものです。


## **2. データとのダンス: モデリングに使用するデータフレームの作成**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>イラスト: @allison_horst</figcaption>


<!--![イラスト: \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.ja.jpg){width="700"}-->


必要なライブラリとデータセットを読み込みます。データを以下の条件を満たすデータフレームに変換します：

- バスケット単位で価格が設定されているカボチャのみを取得する

- 日付を月に変換する

- 高値と安値の平均を計算して価格を求める

- 価格をバスケット単位の数量に基づいた価格に変換する

> これらのステップは[前回のレッスン](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb)で取り上げました。


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

冒険心にあふれて、汚れたデータを調査しクリーニングするためのシンプルな関数を提供する[`janitorパッケージ`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor)を探ってみましょう。例えば、データの列名を見てみましょう:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 私たちはもっと良くできます。これらの列名を [snake_case](https://en.wikipedia.org/wiki/Snake_case) 規約に変換して `janitor::clean_names` を使用して `friendR` にしましょう。この関数について詳しく知るには: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

とてもきれいに整理されましたね 🧹！では、前回のレッスンのように `dplyr` を使ってデータとダンスしましょう！ 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

よくできました！👌 これで、新しい回帰モデルを構築するためのクリーンで整然としたデータセットが手に入りましたね！

散布図を試してみますか？


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


散布図を見ると、8月から12月までの月別データしかないことを思い出させてくれます。線形的に結論を導き出すには、もっと多くのデータが必要かもしれません。

もう一度、モデリングデータを見てみましょう:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

もし、`city` や `package` のような文字型の列を基にしてカボチャの `price` を予測したい場合はどうすればよいでしょうか？ あるいはもっと簡単に言えば、例えば `package` と `price` の間の相関（どちらも数値である必要があります）を見つけるにはどうすればよいでしょうか？ 🤷🤷

機械学習モデルは、テキスト値よりも数値の特徴量を使う方が効果的に動作するため、通常はカテゴリカルな特徴量を数値表現に変換する必要があります。

つまり、モデルが効果的に利用できるように予測変数を再フォーマットする方法を見つける必要があります。このプロセスは `特徴量エンジニアリング` として知られています。


## 3. モデリングのためのデータ前処理 - recipes 👩‍🍳👨‍🍳

予測値をモデルが効果的に利用できるように再構成する活動は、`特徴量エンジニアリング`と呼ばれています。

異なるモデルには異なる前処理の要件があります。例えば、最小二乗法では、月、品種、都市名などの`カテゴリ変数のエンコード`が必要です。これは単に、`カテゴリ値`を持つ列を、元の列の代わりとなる1つ以上の`数値列`に`変換`することを意味します。

例えば、データに以下のカテゴリ特徴量が含まれているとします：

|  city   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

これに*順序エンコーディング*を適用して、各カテゴリにユニークな整数値を代入すると、次のようになります：

| city |
|:----:|
|  0   |
|  1   |
|  2   |

これをデータに適用していきます！

このセクションでは、もう一つの素晴らしいTidymodelsパッケージ：[recipes](https://tidymodels.github.io/recipes/)を探求します。このパッケージは、モデルをトレーニングする**前**にデータを前処理するために設計されています。レシピの中心的な役割は、データセットをモデリングに適した状態にするためにどのステップを適用すべきかを定義するオブジェクトです。

では、予測列のすべての観測値にユニークな整数を代入することで、モデリングの準備をするレシピを作成してみましょう：


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

素晴らしいですね！👏 私たちは、結果（価格）とそれに対応する予測因子を指定し、すべての予測因子の列を整数のセットにエンコードする最初のレシピを作成しました 🙌！では、これを簡単に分解してみましょう：

-   `recipe()`への呼び出しで式を使用すると、`new_pumpkins`データを参照として変数の*役割*をレシピに伝えます。例えば、`price`列は`outcome`の役割に割り当てられ、残りの列は`predictor`の役割に割り当てられています。

-   `step_integer(all_predictors(), zero_based = TRUE)`は、すべての予測因子を0から始まる番号付けで整数のセットに変換することを指定しています。

きっとこう思っているかもしれませんね：「これってすごくクールだけど、レシピが本当に期待通りに動作しているか確認したい場合はどうすればいいの？🤔」

それは素晴らしい考えです！レシピが定義されると、データを実際に前処理するために必要なパラメータを推定し、処理されたデータを抽出することができます。通常、Tidymodelsを使用する場合にはこれを行う必要はありません（すぐに通常の方法である`workflows`を見ていきます）が、レシピが期待通りに動作しているか確認するための簡単なチェックを行いたい場合には役立ちます。

そのためには、もう2つの動詞が必要です：`prep()`と`bake()`。そしていつものように、[`Allison Horst`](https://github.com/allisonhorst/stats-illustrations)による小さなRの仲間たちがこれをよりよく理解する手助けをしてくれます！

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Artwork by @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): トレーニングデータセットから必要なパラメータを推定し、それを他のデータセットに適用できるようにします。例えば、特定の予測子列において、どの観測値が整数の0、1、2などに割り当てられるかを決定します。

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): 準備済みのレシピを使用して、任意のデータセットに操作を適用します。

つまり、レシピを準備して適用することで、予測子列がモデルに適合される前にまずエンコードされることを、内部的に確認することができます。


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

やったね！🥳 処理されたデータ `baked_pumpkins` のすべての予測変数がエンコードされており、定義された前処理手順（レシピ）が期待通りに機能することが確認されました。これにより読みづらくはなりますが、Tidymodelsにとってははるかに理解しやすくなります。どの観測値が対応する整数にマッピングされているか、少し時間をかけて確認してみてください。

また、`baked_pumpkins` は計算を実行できるデータフレームであることも重要なポイントです。

例えば、データの2つのポイント間で良い相関を見つけて、良い予測モデルを構築する可能性を探ることができます。そのために、`cor()` 関数を使用します。`?cor()` を入力して、この関数についてさらに詳しく調べてみてください。


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


実際のところ、CityとPriceの間には弱い相関しかありません。しかし、PackageとそのPriceの間にはもう少し強い相関があります。それは納得できますよね？通常、箱が大きければ大きいほど、価格も高くなるものです。

ついでに、`corrplot`パッケージを使って、すべての列の相関行列を視覚化してみましょう。


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 ずっと良くなりました。

このデータに対して次に尋ねるべき良い質問は、 '`特定のかぼちゃパッケージの価格はどれくらい期待できますか？`' です。さっそく始めましょう！

> Note: **`pumpkins_prep`** という準備済みのレシピを **`bake()`** し、**`new_data = NULL`** を指定すると、処理済み（つまりエンコードされた）トレーニングデータを抽出できます。例えば、テストセットのような別のデータセットがあり、そのデータがレシピでどのように前処理されるかを確認したい場合は、**`new_data = test_set`** を指定して **`pumpkins_prep`** を単純に bake すればよいのです。

## 4. 線形回帰モデルを構築する

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalliによるインフォグラフィック</figcaption>


<!--![Dasani Madipalliによるインフォグラフィック](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ja.png){width="800"}-->


レシピを作成し、データが適切に前処理されることを確認したので、次は回帰モデルを構築して次の質問に答えましょう: `特定のかぼちゃパッケージの価格はどれくらいになるか？`

#### トレーニングセットを使用して線形回帰モデルを訓練する

すでにお気づきかもしれませんが、*price* 列が `結果` 変数であり、*package* 列が `予測子` 変数です。

これを行うために、まずデータを分割し、80%をトレーニングセット、20%をテストセットに割り当てます。その後、予測子列を整数のセットにエンコードするレシピを定義し、モデル仕様を構築します。レシピを準備して焼く必要はありません。データが期待通りに前処理されることはすでに確認済みだからです。


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

よくやりました！これでレシピとモデル仕様が揃ったので、それらを一つのオブジェクトにまとめる方法を見つける必要があります。このオブジェクトは、まずデータを前処理（裏でprep+bakeを実行）、前処理済みデータに基づいてモデルを適合させ、さらに必要に応じて後処理を行うことも可能にします。これで安心ですね！🤩

Tidymodelsでは、この便利なオブジェクトを[`workflow`](https://workflows.tidymodels.org/)と呼び、モデリングの構成要素をうまく保持します！これは、*Python*で言うところの*パイプライン*に相当します。

では、すべてをワークフローにまとめましょう！📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

おまけに、ワークフローもモデルとほぼ同じ方法で適合させたりトレーニングしたりすることができます。


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

モデルの出力から、トレーニング中に学習された係数を見ることができます。これらは、実際の変数と予測された変数の間の全体的な誤差を最小にする最適な直線の係数を表しています。

#### テストセットを使ってモデルの性能を評価する

モデルの性能を確認する時が来ました 📏！どうやってこれを行うのでしょうか？

モデルをトレーニングしたので、`parsnip::predict()`を使ってテストセットに対する予測を行うことができます。その後、これらの予測値を実際のラベル値と比較して、モデルがどれだけうまく（またはうまくいっていないか）動作しているかを評価します。

まずは、テストセットに対する予測を行い、その列をテストセットに結合するところから始めましょう。


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

はい、モデルを訓練し、予測を行いました！🔮 その性能はどうでしょうか？モデルの評価をしてみましょう！

Tidymodelsでは、これを `yardstick::metrics()` を使って行います！線形回帰の場合、以下の指標に注目しましょう：

-   `Root Mean Square Error (RMSE)`: [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) の平方根。この指標はラベル（この場合はカボチャの価格）と同じ単位で絶対的な値を示します。値が小さいほどモデルが良いことを意味します。（簡単に言えば、予測がどれだけ平均的に間違っているかを表します！）

-   `Coefficient of Determination (通常は R-squared または R2 として知られる)`: 相対的な指標で、値が高いほどモデルの適合度が良いことを示します。この指標は、予測値と実際のラベル値の間の分散をモデルがどれだけ説明できるかを表しています。


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

モデルのパフォーマンスが低下しましたね。パッケージと価格の散布図を視覚化し、それに基づいて予測値を使って最適な回帰直線を重ねることで、より良い指標を得られるか試してみましょう。

そのためには、テストセットを準備して処理し、パッケージ列をエンコードした後、それをモデルが出した予測値と結合する必要があります。


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


線形回帰モデルは、パッケージとその対応する価格の関係をうまく一般化できていないことがわかりますね。

🎃 おめでとうございます！いくつかの種類のカボチャの価格を予測するモデルを作成しました。これであなたのホリデー用カボチャ畑は素晴らしいものになるでしょう。ただし、もっと良いモデルを作ることもできるかもしれません！

## 5. 多項式回帰モデルを構築する

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalliによるインフォグラフィック</figcaption>


<!--![Dasani Madipalliによるインフォグラフィック](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ja.png){width="800"}-->


データが必ずしも線形関係を持たない場合でも、結果を予測したいことがあります。多項式回帰は、より複雑な非線形関係に対して予測を行うのに役立ちます。

例えば、カボチャのデータセットにおけるパッケージと価格の関係を考えてみましょう。変数間に線形関係がある場合もありますが（例えば、カボチャの体積が大きいほど価格が高くなる）、これらの関係が必ずしも平面や直線としてプロットできるとは限りません。

> ✅ こちらに、多項式回帰を使用できるデータの[いくつかの例](https://online.stat.psu.edu/stat501/lesson/9/9.8)があります。
>
> 前のプロットで、品種（Variety）と価格（Price）の関係をもう一度見てみてください。この散布図は直線で分析すべきように見えますか？おそらくそうではないでしょう。この場合、多項式回帰を試すことができます。
>
> ✅ 多項式とは、1つ以上の変数と係数から成る数学的表現です。

#### トレーニングセットを使って多項式回帰モデルを訓練する

多項式回帰は、非線形データにより適合するような*曲線*を作成します。

多項式モデルが予測性能を向上させるかどうかを確認してみましょう。以前とほぼ同じ手順に従います：

-   データをモデリングに適した状態にするための前処理手順を指定するレシピを作成します。具体的には、予測変数のエンコードや次数 *n* の多項式の計算などです。

-   モデル仕様を構築します。

-   レシピとモデル仕様をワークフローにまとめます。

-   ワークフローを適合させてモデルを作成します。

-   テストデータでモデルの性能を評価します。

それでは始めましょう！


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### モデルのパフォーマンスを評価する

👏👏ポリノミアルモデルを構築しましたね。それでは、テストセットで予測を行いましょう！


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

ウーフー、`yardstick::metrics()` を使用して test_set に対するモデルのパフォーマンスを評価しましょう。


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 パフォーマンスが大幅に向上しました。

`rmse` は約7から約3に減少しました。これは、実際の価格と予測価格の間の誤差が減少したことを示しています。これを*ざっくり*解釈すると、平均的に誤った予測が約3ドル程度の誤差であることを意味します。`rsq` は約0.4から0.8に増加しました。

これらすべての指標は、ポリノミアルモデルが線形モデルよりもはるかに優れていることを示しています。素晴らしい仕事です！

これを視覚化できるか見てみましょう！


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


データにより適した曲線が表示されましたね！🤩

これをさらに滑らかにするには、`geom_smooth` に多項式の式を渡すことで実現できます。例えば、次のようにします:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

まるで滑らかな曲線のようですね！🤩

新しい予測を作成する方法は次の通りです:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


`polynomial model`の予測は、`price`と`package`の散布図を考慮すると納得できますね！そして、もしこれが前のモデルよりも良いモデルであるなら、同じデータを見て、より高価なカボチャの予算を立てる必要があります！

🏆 よくできました！1つのレッスンで2つの回帰モデルを作成しましたね。回帰の最終セクションでは、カテゴリを決定するためのロジスティック回帰について学びます。

## **🚀チャレンジ**

このノートブックでいくつか異なる変数をテストして、相関がモデルの精度にどのように対応するかを確認してください。

## [**講義後のクイズ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **復習と自己学習**

このレッスンでは線形回帰について学びました。他にも重要な回帰の種類があります。Stepwise、Ridge、Lasso、Elasticnetの手法について調べてみてください。さらに学ぶために良いコースとして、[Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning)があります。

素晴らしいTidymodelsフレームワークの使い方をもっと学びたい場合は、以下のリソースをチェックしてください：

-   Tidymodelsウェブサイト: [Tidymodelsの始め方](https://www.tidymodels.org/start/)

-   Max KuhnとJulia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **感謝の言葉：**

[Allison Horst](https://twitter.com/allison_horst?lang=en) に、Rをより親しみやすく魅力的にしてくれる素晴らしいイラストを作成していただきました。彼女の[ギャラリー](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM)でさらに多くのイラストを見つけることができます。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確さが含まれる可能性があります。元の言語で記載された原文が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤認について、当社は一切の責任を負いません。
